In [1]:
import pandas as pd
import csv
import time 
import pickle
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from metaflow import FlowSpec, IncludeFile, step
import numpy as np
import sys
sys.path.append("/usr/local/lib/python3.9/site-packages")
# I import sys instead of import bs4 
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from time import sleep 
from pandasql import sqldf

pd.set_option('display.max_rows', 10000)

In [2]:
df = pd.read_csv('clean_category.csv')
df = df.drop_duplicates()
df = df.dropna()
df

,username,sold_price,designer,score,transaction,category_1,category_2,category_3,description,sub_title,image_count,size,color,condition,feedback_count
0,shmoterium,30.0,Nike x Streetwear,5.0,1393.0,Menwear,Bottom,Short,Nike Shorts\nSize L\nConditions not very good\...,Vintage Nike Dri Fit Shorts,8,Men's / US 34 / EU 50,Green,Used,441.0
1,ThreepeatThrifting,165.0,Nike,5.0,97.0,Menwear,Footwear,Low Top Sneaker,Nike Dunk Panda Low Size 10 Good Condtion\nOG ...,Nike Dunk Panda Low Size 10 Good Condtion,6,Men's / US 10 / EU 43,Black,Used,58.0
2,street_weather,89.0,Nike x Streetwear,5.0,44.0,Menwear,Outerwear,Light Jacket,Size L\nCondition 8/10\nNeed washing,NIKE SHOX VINTAGE CENTER LOGO OVERSIZE ANORAK ...,7,Men's / US L / EU 52-54 / 3,White,Gently Used,5.0
3,Igotsole503,180.0,Jordan Brand x Nike,5.0,49.0,Menwear,Footwear,Hi Top Sneaker,Mens size 10 / Has original box\n9/10 conditio...,Jordan 6 Electric Green size 10,11,Men's / US 10 / EU 43,Black,Gently Used,36.0
4,usedclothes,51.0,Nike x Stussy,5.0,2240.0,Menwear,Top,Short Sleeve T,Brand new with tags\nTagged medium fits oversi...,Increase the Peace Tee Nike x Stüssy M/L,7,Men's / US M / EU 48-50 / 2,White,New,798.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72205,UNTRUERELIGION,43.0,Calvin Klein x Streetwear,5.0,283.0,Menwear,Bottom,Denim,"In nice worn in condition, accurately represen...",Vintage Calvin Klein Denim Jeans,7,Men's / US 34 / EU 50,Brown,Used,98.0
72206,Markhamvtg,35.0,Calvin Klein x Streetwear,5.0,96.0,Menwear,Bottom,Short,Vintage Denim Calvin Klein Denim Shorts\nWaist...,Vintage Denim Calvin Klein Denim Shorts,4,Men's / US 34 / EU 50,Blue,Used,27.0
72207,shopluckyboy,40.0,Calvin Klein x Streetwear,5.0,293.0,Menwear,Bottom,Short,Calvin Klein swim shorts\nWithout defects\nVer...,Calvin Klein black swim shorts,6,Men's / US 32 / EU 48,Black,Gently Used,111.0
72208,pottymuoth,24.0,Calvin Klein,5.0,485.0,Menwear,Outerwear,Light Jacket,"Very good color. Only worn a handful of times,...",Calvin Klein Waterproof Workers Jacket,5,Men's / US M / EU 48-50 / 2,Orange,Gently Used,231.0


In [77]:
df['trans_group'] = pd.cut(df['transaction'], [-float('inf'), 30, 70, 200, float('inf')], labels=[2, 1, 0, 3])
df['image_count_group'] = pd.cut(df['image_count'], [-float('inf'), 5, 15, 20, float('inf')], labels=[2, 1, 0, 3])
df

,username,sold_price,designer,score,transaction,category_1,category_2,category_3,description,sub_title,image_count,size,color,condition,feedback_count,trans_group,image_count_group
0,shmoterium,30.0,Nike x Streetwear,5.0,1393.0,Menwear,Bottom,Short,Nike Shorts\nSize L\nConditions not very good\...,Vintage Nike Dri Fit Shorts,8,Men's / US 34 / EU 50,Green,Used,441.0,3,1
1,ThreepeatThrifting,165.0,Nike,5.0,97.0,Menwear,Footwear,Low Top Sneaker,Nike Dunk Panda Low Size 10 Good Condtion\nOG ...,Nike Dunk Panda Low Size 10 Good Condtion,6,Men's / US 10 / EU 43,Black,Used,58.0,0,1
2,street_weather,89.0,Nike x Streetwear,5.0,44.0,Menwear,Outerwear,Light Jacket,Size L\nCondition 8/10\nNeed washing,NIKE SHOX VINTAGE CENTER LOGO OVERSIZE ANORAK ...,7,Men's / US L / EU 52-54 / 3,White,Gently Used,5.0,1,1
3,Igotsole503,180.0,Jordan Brand x Nike,5.0,49.0,Menwear,Footwear,Hi Top Sneaker,Mens size 10 / Has original box\n9/10 conditio...,Jordan 6 Electric Green size 10,11,Men's / US 10 / EU 43,Black,Gently Used,36.0,1,1
4,usedclothes,51.0,Nike x Stussy,5.0,2240.0,Menwear,Top,Short Sleeve T,Brand new with tags\nTagged medium fits oversi...,Increase the Peace Tee Nike x Stüssy M/L,7,Men's / US M / EU 48-50 / 2,White,New,798.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72205,UNTRUERELIGION,43.0,Calvin Klein x Streetwear,5.0,283.0,Menwear,Bottom,Denim,"In nice worn in condition, accurately represen...",Vintage Calvin Klein Denim Jeans,7,Men's / US 34 / EU 50,Brown,Used,98.0,3,1
72206,Markhamvtg,35.0,Calvin Klein x Streetwear,5.0,96.0,Menwear,Bottom,Short,Vintage Denim Calvin Klein Denim Shorts\nWaist...,Vintage Denim Calvin Klein Denim Shorts,4,Men's / US 34 / EU 50,Blue,Used,27.0,0,2
72207,shopluckyboy,40.0,Calvin Klein x Streetwear,5.0,293.0,Menwear,Bottom,Short,Calvin Klein swim shorts\nWithout defects\nVer...,Calvin Klein black swim shorts,6,Men's / US 32 / EU 48,Black,Gently Used,111.0,3,1
72208,pottymuoth,24.0,Calvin Klein,5.0,485.0,Menwear,Outerwear,Light Jacket,"Very good color. Only worn a handful of times,...",Calvin Klein Waterproof Workers Jacket,5,Men's / US M / EU 48-50 / 2,Orange,Gently Used,231.0,3,2


In [90]:
pysqldf = lambda q: sqldf(q, globals())

start0 = pysqldf("""
    SELECT df.*, categories,
    case when condition = 'New' then 0 when condition = 'Gently Used' then 1 when condition = 'Used' then 2 else 3 end as condition_encoded,
    case when feedback_count >= 1000 then 0 when feedback_count >= 500 then 1 when feedback_count >= 200 then 2 else 3 end as feedbackcode,
    case when color IN ('Black', 'Blk', 'Dark gray', 'Leopard white black', 'Deep black', 'Black colorblock','Puma black','Blackwatch','Graphite black','White/ black','Violet/black','Heather black','Anthracite black','Coated black','Black polka dot','Black patent','Black/phantom-ale','Black / black','Black / in photo','Black glow in the dark','Ivory black','Black white stripes','Violet lilla black white','Black and plaid','Black sail pinstripe','Cloud cream black','Black sail','Asphalt/ outer space/ black','Black/mirror','Black / milk / milk','Black gun metal','Washed out black', 'Black & white', 'Anthracite / black','Black and ivory','Black/egret','Black and white/cream','Ash black','Black /white','Stone washed black','Transparent black', 'Light black','Waxed black','Black/leopard','Glossy black','Black/black-cone-white','Black skulls','Black reflective','Black jeans','Black gucci signature leather','Black and white diagonal cross','Black/washed','Canvas/black','Black white racing jacket','Grime black','Midstone black','Black cotton', 'All black','Black waxed','Black/bone','Dirty light black','Overdye jet black','Ec black','Sequoia black','Wash black denim','White-black','Triple black','Black violet white','Faded black distressed','Speed graffiti black','Black grey white','Black dye','Black-white','Black /','Ainara/black','Black reversible','Tnf black / tnf black','White/black/white','Almost black/white','Black/sail','Black sugar','Sulphur black','Utility black','Black/whit','Black / metallic copper','Deep black','Black polkadot','Black stripes','Black & true white','Black white acetate','Black paisley','Black colorblock', 'Leopard black wood','Black/black', 'Black and milk','Black wool','Black/egret/black','Fading black','Black/polished black','Black stripe','Black/ white','Black rusty waxed','Faded black wash','Black / printed','Black/white suede','Black with floral jacquard','Dark used black','Black white sleeve','Black on black','Black jean','Black&white','Black bleach','Black mineral','Egret/egret/black','Black/3m','Tnf black','Leopard white black','Smoked black','Black (no.15)','Cream/black/white','Black paint splatter','Black / gum','Black vintage wash denim','Black w/translucent rubber sole','Black white and chalk','Black monogram','Black nova check plaid tartan','Sunfaded black','Faded black (as on photo)','Lily white/egret/black','Black metallic','Black/nova check','Nova check / black', 'Black milk','Black and cream','Destroy black','Vintage wash black','Black python','Black strip','Sun faded black','Double black','Black/canvas','Stressed black and white','Faded black denim','Black white acetate', 'Black/polished black', 'Triple black', 'Faded black', 'Black stripe', 'Heather black', 'Zebra', 'Washed black', 'Blac', 'Black / white', 'Black white', 'Black wash', 'Black/white', 'White/black', 'Black and white', 'White black', 'Black wax', 'Dark', 'Color block', 'Black/cream', 'Vintage black', 'Black multi', 'White / black', 'Black multicolor', 'Faded black', 'Black/white', 'Black white', 'Washed black', 'White/black', 'Black and white', 'Black / white', 'White black','Black/white/cream','Acid wash black','Black 3m','Black and reflective','Used black','Matte black','Whtie/black','Marble black','Black distressed','Black velvet','Blacks','Black ex','Black plaid','Black wax','Black wash','White and black','Black faded','Cream/black','Black/cream','Vintage black','White / black','Black smoke','Black cream','Charcoal black', 'Black pearl','Black violet','Raven black','Translucent black','Rustic black','Black pinstripe','White & black','Core black','Light wash black','Blackwhite stripes','Black/milk','Poetic black') THEN 'Black' 
        when color IN ('White', 'Whte', 'Gae white','White', 'Black white', 'Off white', 'Black and white', 'Offwhite', 'White black', 'Off-white', 'White yellow', 'White and black', 'White cream', 'Cream white', 'Lily white', 'White mix', 'Whisper white', 'Cloud white', 'White asparagus', 'Milky white', 'White yellowed', 'Black yellow white', 'White powder', 'Ivory white', 'White painted', 'White & black', 'White printed', 'Transparent white', 'Vintage white', 'Black & white', 'White bone', 'Blackwhite stripes', 'Sail white', 'Chalk', 'White paint', 'White denim', 'Light cream white', 'Optic white', 'White rainbow', 'Puma white', 'Leopard white black', 'Gae white', 'Black white racing jacket', 'White striped', 'Black white stripes', 'White and cream', 'Black white sleeve', 'Birch white', 'White-black', 'White chalk', 'Owhite', 'White distressed', 'White regrind', 'Orange white', 'Black white yellow', 'Black white and chalk', 'Stressed black and white', 'White sail', 'Black orange white', 'White tie dye', 'Triple white', 'Aspen white', 'White black orange', 'White line', 'Black and white diagonal cross', 'Classic white', 'White milk', 'White plaid', 'Turaqoise white orange', 'Orchid white', 'White silver black', 'Broken white', 'True white', 'White gold', 'Black white acetate', 'Chalk white', 'Black&white', 'Cwhite', 'Sand white', 'Silver white', 'Off white tour', 'Fuschia white', 'White widow', 'Off white', 'Cream / chalk', 'Chalk white', 'Bleached wash', 'Ivory', 'Natural ivory ', 'Milk', 'Pearl', 'Bleach', 'Bone', 'Whie', 'Cream/white', 'Offwhite', 'Clay', 'White/cream', 'Off-white', 'White multi') THEN 'White' 
        when color IN ('Blue', 'Navy', 'Dark blue', 'Navy blue', 'Light blue', 'Blue denim', 'Baby blue', 'Blue distressed', 'Faded blue', 'Blue black', 'Dark navy', 'Washed blue', 'Blue jean', 'Blue white', 'Light blue wash', 'Black blue', 'Sky blue', 'Blue wash', 'Faded navy', 'Denim blue', 'Blue red', 'White blue', 'Indigo blue', 'Deep blue', 'Blue gray', 'Royal blue', 'Blue grey', 'Blue purple', 'Blue jeans', 'Navy green', 'Blue tie dye', 'Light blue jeans', 'Navy red', 'Blue yellow', 'Washed navy', 'Blueblack', 'Soft blue', 'Dark blue denim', 'Blue white red', 'Blu navy', 'Blues', 'Blue green', 'Blue plaid', 'Blue and red', 'Bleu navy', 'Light blue washed', 'Blue navy', 'Gray blue', 'University blue', 'Brown beige blue', 'Blue and black', 'Aqua blue', 'Midnight blue', 'Black navy', 'Powder blue', 'Pale blue', 'Steel blue', 'Blue and white', 'Dark blue white', 'Blue fade', 'Mid blue', 'Light blue denim', 'Green navy', 'Blue faded', 'Grey blue', 'Teal blue', 'Black white blue', 'Blue black distressed', 'Blue orange', 'Medium blue', 'Navy white', 'Navy faded', 'Navy grey', 'Midnight navy', 'Blue orange yellow', 'Dirty blue', 'Dark navy black', 'Yellow and navy blue', 'Navy black', 'Raw blue', 'Pastel blue', 'Blue red white', 'White faded blue', 'Navy cream', 'Tinted blue', 'Navy blue white red', 'Jeans blue', 'Sax blue', 'Blue stripe', 'White blue red', 'Blue marine', 'Electic blue', 'Gray navy', 'Vintage blue', 'Blue striped', 'Green blue', 'Blue pink', 'Bright blue', 'Sea blue', 'White and blue', 'French navy', 'Blue and purple', 'Dusty blue', 'White navy', 'Red dark blue', 'Washed dark blue', 'Blue navy purple', 'Blue indigo', 'Navy yellow', 'Purple blue abstract floral pattern', 'Shiny blue', 'Plaid blue and black', 'Greenish blue metal', 'Black & blue', 'Blue with brown checked', 'Blue navy black', 'Royal blue white', 'Smoky blue', 'Green blue yellow', 'Black and navy', 'Beige navy red', 'Blue washed denim', 'Faded black blue', 'Blue - turquoise blue', 'Navy blue white', 'Marble wash blue indigo', 'Navy paisley print', 'Marine blue', 'Washed baby blue', 'Black and blue', 'Teal navy', 'White blue green', 'Frozen blue', 'White yellow blue', 'Turquoise blue', 'Navy khaki', 'Blue ripped', 'Red & blue', 'Blue-white red line', 'Deep royal blue', 'Forest green yellow blue', 'White blue yellow', 'Pink and navy', 'White red blue', 'Runners blue', 'Navy white red', 'Stripe blue', 'Black yellow orange red blue', 'Blue black red', 'Green navy red', 'Blue purple faded', 'Blue ￼', 'Navy denim', 'Og white blue', 'Day blue', 'Blue flannel', 'Navy checker', 'Yellow blue', 'White blue orange', 'Black, gray, blue', 'Black red blue white', 'Navy tie dye', 'Blue tartan', 'Iridescent blue', 'Navy beige', 'Green navy blue', 'White blue pink green', 'Carolina blue', 'Dark blue wash', 'Blue beige', 'Striped blue', 'Black , white and sky blue', 'Bark blue', 'Navy tan', 'Powder blue white', 'Rusty faded blue', 'Yellow and blue', 'Blue red black', 'Mid blue trash', 'Navy and green', 'Blue bleach', 'Beige and blue', 'Blue and grey', 'Blue stripes', 'Blue white stripes', 'Racer blue', 'Top blue on navy check frame & dark grey lenses', 'Dark blue jeans', 'Purple pink blue', 'Dust blue', 'Light stone wash blue', 'Green blue red', 'Rusty dark blue', 'Grey and blue', 'Black purple blue orange', 'White navy red', 'Obsidian navy', 'Caspian navy', 'Cream white blue orange', 'Navy & beige', 'Dark blue tie dye', 'Blue navy grey', 'Blue and navy', 'Light blue trash', 'Blue and colorful', 'Dark blue & grey', 'Blue ultramarine', 'Navy yellow blue', 'Light blue and black', 'Slate blue', 'Surf the web blue', 'Summer blue', 'Navy stripe', 'Olive blue', 'Blue navy gray', 'Shabbyblue', 'Navy indigo', 'Pioneer blue', 'Blue, white', 'Blue check', 'Blue tint', 'Columbia blue', 'Blue, yellow, pink, green', 'Orange blue', 'Green and navy', 'Blue white green brown black', 'White black blue', 'Blue ocean', 'Navy blue - refer to picture', 'Deep navy', 'Gold and blue', 'Orange dark blue', 'Pearl blue', 'Blue celeste', 'Black or navy', 'Very dark blue', 'Blue zebra', 'Red blue yellow', 'Blue silver', 'Blue white tiedye', 'Blue mix', 'Crazy fades blue black', 'Ombre blue', 'Navy and blue', 'Colonial blue', 'White with royal blue', 'Blue light wash', 'Navy blue, white', 'Puma black - blue turquoise', 'Blue and yellow', 'Hi res blue', 'Beige navy', 'Grey blue tan', 'Navy color', 'Navy purple', 'Light blue baby blue', 'White, red, blue, gold', 'Navy gray', 'Blue brown', 'Yellow red blue', 'Two tones navy', 'Naval navy', 'Midnighht blue', 'Red and blue', 'Ivory, blue', 'Black blue green', 'Red yellow navy', 'Blue navy silver', 'Old dark blue', 'Navy red green', 'Electric blue', 'Blue marble wash', 'Military blue', 'Blue & brown', 'Black brown navy', 'Brown blue flannel', 'Blue & black', 'Light blue & white', 'Blue black green', 'Navy plaid', 'Navy pinstripe', 'Black blue red', 'Gray blue orange', 'Blue yellow white', 'Navy blue burgundy', 'Uniform blue', 'Mid denim blue', 'Clay blue', 'Light gray blue', 'Grey-blue', 'Tie dye blue', 'Red navy', 'White and navy blue', 'Navy and red', 'Blue white stripe', 'Grey navy', 'Varsity blue', 'Navy blue purple white', 'Gold navy', 'Yellow white navy', 'Pink red blue colorful yellow', 'Red green navy blue', 'Blue velour', 'Blue delavè', 'Denim blue jeans', 'Medium wash blue', 'Ice blue', 'Black&blue', 'Vintage blue denim', 'Navy with black', 'Grey black blue', 'Dark blue red', 'Blue suede', 'White green blue', 'Faded dark blue', 'Blueberry', 'Distressed faded denim blue', 'White turn blue skies', 'Washed light blue', 'White with royal blue', 'Blue black', 'Black denim', 'Dark navy black', 'Puma black - blue turquoise','Black or navy','Crazy fades blue black','Blue navy black','Black / navy','Denim black','Black and blue','Black / teal','Black washed denim','Navy with black','Plaid blue and black','Black&blue','Faded black blue','Teal black','Black indigo','Black and navy','Black denim/leather','Blue / black','Navy black','Light blue and black','Black teal','Black & blue','Dark navy/black','Blue & black','Dark denim and black','Black / royal blue','Black blue','Blue/black','Navy/black','Blue black distressed','Black / blue','Blueblack','Black navy','Blue and black','Black/navy','Black/blue', 'Blue black', 'Bleu marine', 'Denim ocean', 'Black & blue', 'Rustic raw indigo denim', 'Marble wash blue indigo', 'Black and navy', 'Light blue trash', 'Aqua/blue', 'Blue / gray', 'Blue, white', 'Navy/white', 'Uniform blue', 'Blue gray', 'Blu navy', 'Bleu navy', 'Light blue jeans', 'Denim', 'Light denim', 'Blue jeans', 'Blue grey', 'Dark denim', 'Faded denim', 'Black / blue', 'Grey/blue', 'Royal blue', 'Jeans', 'Denim/tan', 'Black denim', 'Washed denim', 'Indigo blue', 'Deep blue', 'White blue', 'White/blue', 'Dark blue', 'Blu', 'Navy blue', 'Indigo', 'Light blue', 'Blue denim', 'Baby blue', 'Blue distressed', 'Faded blue', 'Dark navy', 'Washed blue', 'Blue jean', 'Blue black', 'Charcoal', 'Black/blue', 'Blue/white', 'Light blue wash', 'Blue white', 'Light wash', 'Turqoise', 'Turquoise', 'Aqua', 'Blue/navy', 'Black blue', 'Bleu', 'Sky blue', 'Blue wash', 'Faded navy', 'Denim blue', 'Stonewash', 'Blue/black') THEN 'Blue' 
        when color IN ('Grey', 'Gray', 'Ash','Grey', 'Gray', 'Dark grey', 'Washed black', 'Dark gray', 'Black grey', 'Black gray', 'Light grey', 'Heaher gray', 'Light wash', 'Heather gray', 'Washed denim', 'Charcoal gray', 'Stonewash', 'Light gray', 'Heather grey', 'Ash grey', 'Washed gray', 'Black wash', 'Ash gray', 'White grey', 'Gray green', 'Faded grey', 'Charcoal grey', 'Grey black', 'Grey green', 'Grey white', 'Black white grey', 'Gray white', 'Ash', 'Grey mix', 'Grey marl', 'Fade to grey', 'Ash heather', 'Indigo wash', 'Gray black', 'Washed brown', 'Washed grey', 'Cement grey', 'Slate grey', 'Acid wash black', 'Black ash', 'Grey pearl', 'Mist grey', 'Acid wash', 'Top gray', 'Medium wash', 'Reflective grey', 'Grey brown', 'Vintage washed denim', 'Gray to green', 'Red grey', 'Khaki grey', 'Soft grey', 'Grey gray', 'Graphite grey', 'Purple grey', 'Pearl grey', 'Stone washed black', 'Dust grey', 'Ash black', 'White gray', 'Melange gray', 'All grey', 'Washed indigo', 'Gray marl', 'Black & gray', 'Grey red white', 'Smoky gray', 'Gray silver', 'Mud wash', 'Ash tan', 'Green grey', 'Stone wash', 'Military grey', 'Stonewashed denim beige brown', 'Dirty wash', 'Anthracite grey', 'Light wash black', 'Milk grey', 'Washed purple', 'Light washed denim', 'Grey off white cream beige gray', 'Grey plaid', 'Grey with black', 'Leopard print grey', 'Silver with grey, cream and black', 'Grey orange', 'Vintage wash', 'Black dark grey', 'Grey and white', 'Heather grey gray', 'Dark greys', 'Rusty washed', 'Vintage wash denim', 'Burgundy grey', 'Dark grey black', 'Dark grey and red', 'Grey & red', 'Dark grey melange', 'Wash denim', 'Washed out black', 'Marble wash', 'Grey nickel', 'Cool grey', 'Copper wash gold haze', 'Stonewashed brown duck', 'Dark grey white', 'Grey god', 'Steel grey', 'Slate gray', 'Light stone washed', 'Beige grey pale', 'Dusk grey', 'Dark grey tshirt', 'Faded black wash', 'Light brown stonewash', 'Gray transparent', 'Grey or black', 'Dark grey wash', 'Grey red black white', 'Grey purple', 'Mash eye', 'Ash pearl', 'Palmer grey', 'Grey red', 'Orange,black,grey,silver', 'Wash black denim', 'Ash black denim', 'Light grey glow in the dark', 'Gray olive', 'White gray dotted', 'Grey yellow', 'Neutral gray', 'Greenish grey', 'Black charcoal grey', 'Vintage gray', 'Grey and beige', 'Iron gray', 'Black, dark grey', 'Gray brige', 'Vintage wash black', 'Dim gray', 'Black grey distressed', 'Light grey-neutral', 'Black washed denim', 'Black grey washed', 'Lightwash denim', 'Warm grey', 'White grey green', 'Grey and black', 'Grey neon', 'Washed dark grey', 'Black grey white', 'Pink grey', 'Wash dark', 'Black and grey', 'Light grey melange', 'Grayish beige', 'Grey stripes', 'Brown grey', 'Grey antracite', 'Silver gray', 'Faded gray', 'Light grey marl', 'Orange gray', 'Sail neutral grey', 'Grey?', 'Light wash denim', 'Silver grey', 'Melange grey', 'Gray yellow', 'Lightwash', 'Grey red black', 'Stonewash denim', 'Cream grey', 'Light stone wash', 'Dark brownish grey', 'Athletic grey', 'Purple gray', 'Grey pink', 'Clay grey', 'Vintage grey', 'Mudwash', 'Grey nylon', 'Grey-black', 'Dove grey', 'Dark wash', 'Greenish wash', 'Ash silver', 'White and grey', 'Suede grey', 'Distressed grey', 'Washed green', 'Light grey heather', 'Bleached wash', 'Washed khaki', 'Metallic grey', 'Grey chine', 'Washed yellow', 'Stonewash teal', 'Light grey mix', 'Grey white black', 'Middle gray', 'Washed desert grey', 'Ash green', 'Black vintage wash denim', 'Dark grey', 'Black / grey','Black dark grey','Dark gray/black','Black / gray','Marble black / grey','Grey or black','Black charcoal grey','Black grey distressed','Offwhite cream light grey gray black logos','Grey with black','Black grey washed','Grey-black','Double black and grey gray','Black / black grey','Dark grey black','Black, dark grey','Shale / black','Faded black/gray','Faded black/grey','Black & gray','Grey / black','Gray / black','Grey and black','Black and grey','Grey gray black','Black grey','Black/grey','Black white grey','Grey black','Grey/black','Dark grey / black','Gray black','Gray/black','Black ash', 'Black gray', 'Black/gray', 'Washed desert grey', 'Light grey heather', 'Grey and blue', 'Black / gray', 'Ash gray', 'Faded grey', 'Gray/white', 'White grey', 'Ash grey', 'Grey black', 'Dark gray', 'Charcoal grey', 'Black grey', 'Washed gray', 'Black gray', 'Light grey', 'Heather grey', 'Heaher gray', 'White/grey', 'Black/gray', 'Black/grey', 'Heather gray', 'Charcoal gray', 'Light gray') THEN 'Grey'
        when color IN ('Brown', 'Browns', 'Bronze', 'Dinge', 'Reverse mocha', 'Tan aztec', 'Tan/brown/yellow', 'Black/brown','Black brown','Black/tan','Black / brown','Beige/brown/black','Black khaki','Black, beige','Black/dark mocha-university red-sail','Black / light brown','Faded black/brown','Brown', 'Beige', 'Khaki', 'Tan', 'Light brown', 'Dark brown', 'Khaki green', 'Browns', 'Brown tan', 'Light khaki', 'Tan mix', 'Bronze brown', 'Black brown', 'Brown leather', 'Faded brown', 'Mocha brown', 'Light tan', 'Green tartan', 'Chocolate brown', 'Green brown', 'Light beige', 'Beige brown', 'Black beige', 'Tobacco brown', 'Old khaki', 'Havana brown', 'Brown white', 'Red tartan', 'Sand beige', 'Beige khaki', 'Brown faded', 'Cream brown', 'Coffee brown', 'Dark tan', 'Brown green', 'Brown orange', 'Coco brown', 'Army khaki', 'Brown check', 'Brown monogram', 'British khaki', 'Brown sugar', 'Dark khaki', 'Tan brown', 'Brown nude', 'Beige check', 'Golden brown', 'Beige black', 'Camel brown', 'Orange brown', 'Chestnut brown', 'Brown red', 'Clay brown', 'Khaki brown', 'Oak brown', 'White beige', 'Dark beige', 'Brown black', 'Neutral beige', 'Tan aztec', 'Dark birch brown', 'Golden beige', 'Black, beige', 'Black, green, beige', 'Brown striped', 'Rare brown', 'Khaki black', 'Beige ebony monogram', 'Yellowish beige', 'Baroque brown', 'Nest brown', 'Fader brown color', 'Brown purple', 'Brown and orange', 'Black white brown', 'Yellow and brown', 'Saturated khaki', 'Black tan', 'Duck brown', 'Brown and turquoise', 'Brown silver', 'Tanish', 'Off white beige tan sand', 'Antique brown', 'Hemp brown', 'Brown suade', 'Archaeo brown', 'Brown ale', 'Brown beige ￼', 'Bleu brown', 'Light brown, tan', 'Tan white', 'Brown sail', 'Tangerine', 'Olive brown', 'Velvet brown', 'Khaki cream', 'Green and brown', 'Light tartan millerain', 'Whiskey brown', 'Tartan', 'Natural tan', 'Stone khaki', 'White brown green', 'White brown', 'Faded brown black', 'Red beige', 'Distressed brown', 'Brown suede', 'Gold natural brown', 'Black brown silver gold', 'Striped beige', 'Olive (close to brown)', 'Choco brown', 'Olive tan', 'Green khaki', 'Caramel brown', 'Beige and red', 'Mustard brown', 'Chequered brown', 'Birch brown', 'Natural beige', 'Black nova check plaid tartan', 'Brown olive green khaki', 'Olive khaki', 'Havana honey brown', 'Dark khaki green', 'Beige white', 'Burgundy brown', 'Browne', 'Brown corduroy', 'Black khaki', 'Brown & black','Beige/black','Black/ light tan','Black biege','Brown & black','Black coffee','Black / khaki','Black/beige','Cream light beige/black','Brown/black','Black/brown monogram','Khaki black','Faded brown black','Black / burnt sienna','Beige/black glitter','Beige / black','Black/muslin','Black beige','Black / beige','Black tan','Brown / black','Beige black','Black/khaki','Brown black','Fader brown color', 'Black/dark mocha-university red-sail', 'Drab', 'Olive tan', 'Light brown stonewash', 'Mocha br', 'Bianchetto', 'Beige', 'Nipple', 'Nude', 'Beige / brown', 'Black brown', 'Brown / beige', 'Brown leather', 'Bronze brown', 'Tan/beige', 'Mocha', 'Chocolate', 'Leather', 'Tortoise', 'Oatmeal', 'Tan', 'Light khaki', 'Brown tan', 'Taupe', 'Khaki', 'Kaki', 'Light brown', 'Dark brown', 'Black/brown', 'Biege', 'Camel', 'Wheat') THEN 'Brown'
        when color IN ('Green', 'Olive', 'Dark pine', 'Green', 'Olive', 'Olive green', 'Army green', 'Dark green', 'Teal', 'Forest green', 'Green olive', 'Miltary green', 'Military green', 'Faded green', 'Neon green', 'Black green', 'Sage green', 'Light green', 'Emerald green', 'White and green', 'Mint green', 'Bottle green', 'Dark olive', 'White green', 'Green black', 'Aqua green', 'Green white', 'Olive drab', 'Moss green', 'Mix green', 'Green army', 'Pine green', 'Cream green', 'Green purple', 'Acid green', 'Hunter green', 'Seafoam green', 'Yellow green', 'Green print', 'Dark olive green', 'Teal green', 'Jolly green', 'Light pastel green', 'Gold and green', 'Army olive green', 'Glow green', 'Green plaid', 'Dark teal', 'Green and white', 'Sea green', 'Red yellow green', 'Faded olive', 'Swamp green', 'Light olive', 'Green red', 'Green olive military', 'Pale green', 'Pool green', 'Green croc', 'Green & yellow', 'Milk green army colour', 'Black and green', 'Black olive', 'Olive green.', 'Vintage green', 'Pro green', 'Military olive', 'Old olive', 'Bright green', 'Fluo green', 'Black green purple', 'Deep green corduroy', 'Tie dye green', 'Teal black', 'Green and pink', 'Green military', 'Fresh green', 'Ivy green', 'Teal vancouver grizzlies', 'Green bay', 'Industrial green', 'Persian green', 'Plaid green', 'Faded military green', 'Light green spark', 'Yellow olive green', 'Pea green', 'Blackish green', 'Light teal', 'Soft green', 'Olive green and black', 'White green green', 'Green smoke', 'Green red white', 'Green amired', 'Lime green', 'Dark sage green', 'Turquoise green', 'Faded pine green', 'Faded dark green', 'Some shade of green', 'Black and olive', 'Sun faded green fatigue', 'White & green', 'Deep green', 'Green - refer to picture', 'Red and green', 'Olive plaid', 'Green - neon', 'Light sage green', 'Green with black', 'Cream to green', 'Green zebra', 'Green cream', 'Kelly green', 'Eucalyptus green', 'Floral green', 'Green yellow', 'Black teal white', 'Olive mustard', 'Mint green eucalyptus', 'Dark military green', 'Fir green', 'Forest green orange', 'Green,white', 'Alpha green', 'Green neon', 'New olive','Yellow olive green', 'Deep green corduroy','Black/green','Black green','Black and green','Olive green and black','Black/olive green','Light olive/black','Black/pine green','Black/lime green','Black/olive','Green and black','Pine green/black','Blackish green','Green/black','Green black','Black / green','Green / black','Black and olive','Black olive','Green with black','Black/green striped', 'Black/olive', 'Black and olive', 'Forest', 'Dark olive', 'Lime', 'Gray green', 'Grey green', 'Bottle green', 'White and green', 'Blue green', 'Navy green', 'Mint green', 'Green/brown', 'Brown/multicolor', 'Emerald green', 'Black green', 'Green/black', 'Sage green', 'Olive green', 'Neon green', 'Light green', 'Black/green', 'Army green', 'Faded green', 'Dark green', 'Teal', 'Forest green', 'Green olive', 'Khaki green', 'Mint', 'Gren', 'Military green', 'Miltary green', 'Green/white') THEN 'Green'
        when color IN ('Red', 'Black/red', 'Red', 'Maroon', 'Burgundy', 'Black red', 'Red white', 'Checkered', 'Black red white', 'Red black', 'Burgundy red', 'Faded red', 'White red', 'Dark red', 'As pictured', 'Black and red', 'Blood red', 'Light red', 'Reds', 'Red velvet', 'Red gold', 'Dark maroon', 'University red', 'Plaid red', 'Red cream', 'Black white red', 'Brick red', 'Cardinal red', 'Red mix', 'Red striped', 'Red white black', 'Bred', 'Red plaid', 'Lava red', 'Lacquered black gold', 'Black red pinstrip', 'Vintage off white classic red hoodie', 'Sun faded red', 'Red check', 'Weathered rock', 'Red yellow white', 'White red black', 'Red cherry white', 'Red faded', 'Burgundy white', 'Rust red', 'Maroon cream', 'Bordeaux red', 'Maroon pink', 'Fire red', 'Cherry red polka dots', 'Siren red', 'Black-eggnog-red', 'White black red', 'Red plum', 'White yellow red', 'Radish red', 'Photo print muti colored', 'Red and white', 'Red pinstripe', 'Red maroon', 'White red mint', 'Red orange', 'Red and black', 'Burgundy yellow', 'Red tie die', 'Pictured', 'Dyed & dirty red', 'Fade burgundy', 'Red jaw', 'Red white checked', 'Black red purple white', 'Deep red', 'Purple red', 'White maroon', 'Black red yellow', 'Peach red', 'Engine red', 'Red color', 'Dark maroon wood', 'Drkred', 'Red violet', 'Floral red', 'Red rerrari', 'Red crimson','Black red', 'Deep red', 'Maroon', 'Wine','Maroon/black','Black cordovan','Black-eggnog-red','Red/ black','Core black/red', 'Black maroon','Black red pinstrip', 'Black / og burgundy', 'Red and white','Red/black','Brick red/ black','Red and black','Red / black suede','Black/speed red-clear','Black & oxblood','Red black','Grey red black','Black and red','Black/red stripe','Black / red','Black/ red','Black / burgundy', 'Red orange', 'Cream/red', 'Faded red', 'Dark red', 'White red', 'Gray/red', 'Burgundy red', 'Red black', 'Burgundy', 'Burgandy', 'Red/white', 'Black/red', 'White/red', 'Black red', 'Red white', 'Red/black') THEN 'Red'
        when color IN ('Rainbow', 'Green/red', 'White/navy/red','Multicolor', 'Multi', 'Multiple', 'Multicolour', 'Multicoloured', 'White/multi', 'Black/multi', 'White multi', 'Brown/multicolor', 'Multicolored', 'Multi color', 'Black multi', 'Black multicolor', 'Grey multi', 'Cream/multi', 'Gray/multi', 'Blue multi', 'Green/multi', 'White multicolor', 'Blue multicolor', 'Multi-color', 'Purple/multi', 'Navy/multi', 'Multicolored inserts', 'Red/multi', 'Multi / white', 'Multi/purple', 'Blue/multi', 'Yellow/multi', 'Multi gray', 'Red/navy/multicolor', 'Blue/multicolor', 'Multi grey', 'Light beige/multi', 'Khaki/multicolor', 'Pink/multi', 'Multi / blue', 'Black/red/white/multi', 'Black/sliver/multi', 'Greenish gray/multi', 'Multicolor hawaiian', 'Beige/multi', 'Light greenish blue/multi', 'Multicolor/ silver', 'Orange / multi / floral', 'Brown/multi', 'Multi burgundy wine cream offwhite navy', 'Multicolor striped', 'Multicolor yellow', 'Multi colour', 'Multi/tan/black', 'Navy/black/white/multi', 'Black/gray/multi', 'White/teal/multi', 'Goji berry multi', 'Floral/multi', 'Black/multicolor', 'Multicolor￼', 'Multi/denim', 'Multi/translucent gum', 'Multi pastel', 'White multicolour', 'White and multicolor', 'Blue/white/multi', 'Multicolor dark', 'Copper orange/multi', 'White multicolore purple mustard red green blue', 'Cream white/multi', 'Black / multi', 'Multi coloured', 'Multicolor black', 'Denim multicolor', 'White purple multicolor', 'Black/off-white/multi', 'Navy multi check', 'Light gray/multi', 'Multi-color/muted bronze', 'Green/red/orange/multi', 'Orange multi', 'Red / multi color', 'Dark gray / neon multi', 'Gray/navy/multi', '2010 bape multi embroidered denim shorts small nigo asnka', 'Denim multi', 'Black/red/white/blue/multi', 'Cappuccino/maroon/multi-color', 'Blue/navy/white/multi', 'White/red/multi','Black/multi','Black/white/blue','Orange/blue black','Red/black/white','Yellow / grey / black','Black / multi','Black yellow green blue','Beige/black/blue','White/black/red','Navy / black / white','Black orange green','Black/off-white/multi','White black red','Bright citron/black-summit white','White/brown/black','Black red blue white','Black/white/heather grey','Green/brow,black,white','Black/white/gray /blue','White, red & black','Black orange white','Black white brown','Green / black / white','Blue black red', 'Grey black blue','Black/sliver/multi','White/red/black','Grey white black','Black/multicolor','Dark green black beige white burgundy','White/blue/pink/black/red','Navy/black/white','Ftwwht/cblack/ftwwht','Black, rainbow 🌈','Black , white and sky blue','Black/blue/red','Black/egret/white','White silver black','White black orange','Black/gray/white','Turquoise/pink/black','Black/purple-white','Black white orange brown beige tan','Faded black/red/blue','Wine/black-white','Black blue green','Black, gray, blue','Black red yellow','Faded black/green/white','Black/blue/black','Black/red/blue','Black/red/white/multi','Beige/red/black','Red/white/black','Black/blue/silver','Grey red black white','Green/ black/grey','Black/red/white/blue/multi','Yellow/black/red/green','Darkblue / black / red / green','White red black','Black smoke violet orang','Perple/yellow/blue/green/black', 'Black / gray / green','Grey orange black', 'Black/whitenoir fuchsia laser','Black white red beige and red','Blue/black/white/gray','Blue black green','Black red purple white','Crystal white/core black/bright cyan','Ash black denim','Black/teal/green/grey','Black purple blue orange','White/grey/black/red','Reed camo brown / green / black','Black teal white','Red black white','Silver with grey, cream and black','Orange,black,grey,silver','Black/green/yellow/red','Black brown navy','Blue white green brown black','Black white yellow','White/black/cone','Blue red black','Black/brown/red','Black, green, beige','Blue white black','Black green purple','Blue/black/white','Black/green/yellow','Multicolor black','Black/red/white','Navy/black/white/multi','Brown / beige / white / black','Black / grey / white','White/ gold/ black','Black / grey / green plaid','Black yellow orange red blue',  'Green / black / white / yellow','Multi/tan/black','Black / white/ green','Black/grey/orange','Red/black/tan','Black/green/red','Black / blue / orange','Black/blue pink gradient','Black grey gray blue','Black/yellow/white','White/orange/black','Brown black burgundy white','Grey/white/black/red/blue','Black blue red','Black/grey/white','White/yellow/black','Gray/navy/black', 'Black/bordeaux/navy','Black/gray/multi','Black/white/orange','White/cblack/red','Black yellow white','Black/red/purple', 'Black / blue / pink','White/black/grey','White / black / blue','Gray/black/navy/mult','Black / white / red','White black blue','Black / white / green','Yellow / blue / black','Black/white/blue/red','Red white black','Black purple white','Black white red','Black / white / egret','Black/green/cream/white/green','Black/sport red white','Black/white/yellow','Blue quartz/black/grey','Black / white / pink','Black red white','Black/red/grey','Black multi','Grey/white/black/red','Green/black/white','Black white blue','Black multicolor', 'Black/white/red','Black/blue/white','Black/purple/yellow','Black/white/tan','Black mix','Red/yellow/black', 'Olive / grey', 'Red yellow white', 'Purple/gold', 'Colo', 'Gray/navy/black', 'White blue yellow', 'Navy/grey/red/white', 'Beige and red', 'White/black/cone', 'Tri-color', 'Multicolor hawaiian', 'Barbados cherry / whisper white / beige', 'White red black', 'Black/brown/red', 'French tri colour', 'Cream/green/gum', 'White black orange', 'Faded black/green/white', 'Barbados cherry / whisper white / beige', 'Colourful', 'Blue/white/red', 'Blue/orange', 'Green/yellow', 'Navy red', 'Navy/yellow', 'Multi color', 'Blue yellow', 'Multicolored', 'Navy/red', 'Tan mix', 'Blue/yellow', 'Red/yellow', 'Blue/red/white', 'Black red white', 'Black/multi', 'Monogram', 'Checkered', 'Leopard', 'Neon', 'White/multi', 'Red/blue', 'Camouflage', 'Multicoloured', 'Multicolor', 'Multicolour', 'Multi', 'Mix', 'Multiple', 'Stripe', 'Hickory stripe', 'Striped', 'Plaid', 'Mutli', 'Floral', 'Mixed', 'Blue/red', 'Blue red') THEN 'Multicolor'
        when color IN ('Yellow', 'Black/yellow', 'Cream', 'Multicolor yellow', 'Mustard and creme','Black yellow','Black / yello','Light yellow black','Yellow/black','Black and yellow','Black / yellow','Yellow black','Yellow / black','Black, yellow','Sand/black', 'Yellow/re', 'Sweet creme', 'Sand', 'Yellow gradient', 'Ecru', 'Oil yellow', 'Peach', 'Cheetah', 'Black/yellow', 'White/yellow', 'Mustard', 'Black yellow', 'Light yellow') THEN 'Yellow'
        when color IN ('Pink', 'Light pink', 'Coral','Light pink', 'Baby pink', 'Hot pink', 'Black pink', 'Rose pink', 'Pink white', 'Dust pink', 'Pale pink', 'Pink tie dye', 'Dark pink', 'Soft pink', 'Pink and white', 'Pinks', 'Light peach', 'Pink nectar', 'Pink purple', 'Pink milk', 'Pinky', 'Salmon pink', 'Fleur pink', 'Black and pink', 'White cream pink', 'Black salmon', 'Heather pink', 'White purple pink', 'Orange peach', 'Madras pink', 'Supreme burberry rugby pink', 'Mauve pink', 'Raspberry pink', 'Pink cotton candy', 'Vintage champion spell out white and pink ringer crewneck', 'Blush pink', 'Pink cage', 'Pink records', 'Salmon', 'Black/pink','Black/pink','Black / pink','Pink / black','Black pink', 'Pink/black','Black and pink','Black fuchsia','Black salmon', 'Mauve pink', 'Light peach', 'White cream pink') THEN 'Pink'
        when color IN ('Purple', 'Dull grape', 'Grape','Purple', 'Violet', 'Dark purple', 'Light purple', 'Black purple', 'Lilac', 'White purple', 'Purple yellow', 'Purple black', 'Purple white', 'Black purple white', 'Yellow purple', 'Black violet', 'Violet white', 'Faded purple', 'Faded purple white', 'Light violet', 'Grape', 'Violet iridescent', 'Black smoke violet orang', 'Deep purple', 'White and soft purple', 'Gold and violet', 'Prune purple', 'Violet lilla black white', 'Purple stripe', 'Dull grape', 'Tillandsia purple', 'Yellow lilac glow', 'Black and purple', 'Plum purple', 'Purple taupe', 'Violet faded', 'Court purple', 'Violet', 'Lilac', 'Purple/burgundy','Black/purple','Black and purple','Black plum','Black purple','Black / purple','Purple black','Purple/black','Purple / black', 'Light purple', 'Dark indigo', 'Dark purple', 'Black/purple', 'Purple/white', 'Plum', 'Blue purple', 'Black purple', 'Lavender') THEN 'Purple'
        when color IN ('Orange', 'Black/orange', 'Black/orange', 'Black orange','Black, orange','Orange/black','Black / orange','Black orange', 'Orange / white', 'Brazen orange', 'White / orange') THEN 'Orange'
        when color IN ('Silver', 'Black/silver','Black/silver', 'Black / silver','Silver and black','Black silver','Black and silver','Phantom black','Silver metallic / black','Black, silver', 'Palladium') THEN 'Silver'
        when color IN ('Gold', 'Black/gold', 'Black gold', 'Black/gold','Lacquered black gold','Black gold','Black / gold','Yellow gold/black','Black and gold','Black, gold','Gold/black','Teal/gold') THEN 'Gold'
        when color IN ('Tie dye', 'Blue tie dye', 'Black tie dye', 'Tye dye black','Black tie dye', 'Tye dye') THEN 'Tie dye'
        when color IN ('Camo', 'Red camo','Green camo','Camouflage','Red camo','Blue camo','Black camo','Woodland camo','Desert camo','Snow camo','Purple camo','Pink camo','Yellow camo','Tiger camo','Digi camo','Chocolate chip camo','Gray camo','Orange camo','City camo','Reflective camo','Reed camo brown / green / black','Camouflage chocolate chip','Dark camo','Kaki camouflage','Multicamo','Star camo - black/white/olive','Blue chocolate chip camo','Brown camo','White camo','Grey digital camo','Gray/green camo','Black / camo','Camo/neon green','Cow camo','White/camo','Trebark camo','Black/white camo','Black, green camo','Grey/camo','Navy camo / balck','Chocolate camo','Bape line camo','1st camo','Abccamo','Quilted camo','Camouflage army','Tree camo','Black red camo', 'Green camo', 'Black camo', 'Black red camo', 'Star camo - black/white/olive','Black / camo','Black, green camo','Black/white camo','Black camel','Black camo', 'Woodland camo', 'Blue camo', 'Realtree', 'Trebark camo', 'Dark camo',  'City camo', 'Camouflage', 'Woodland camo','Desert camo','Snow camo','Yellow camo','Tiger camo','Digi camo','Chocolate chip camo','Orange camo','Chocolate camo','Camouflage chocolate chip','Dark camo','Abccamo','Trebark camo', 'City camo', 'Kaki camouflage', 'Tree camo', 'Cow camo', '1st camo', 'Camouflage army','Quilted camo', 'Bape line camo', 'Reflective camo') THEN 'Camo'
        when color IN ('Dust', 'Dark dust') THEN 'Dust'
        when color IN ('Checker', 'Novacheck', 'Nova check', 'Check') THEN 'Checker'
        when color IN ('Other', 'Yellow lilac glow', 'Hype', 'At photo', 'Monogram eclipse', 'Rayon silk', 'Gigachadtradprepsexcoreaesthet', 'Jacquard dark', 'Others', '-', '---', 'X', 'N/a', 'No colour', 'Not specified', 'Transparent', 'Refer', 'All over', '?', 'Refer pic', 'On photo', 'See pictures', 'Picture', 'As the pictures', 'See the pictures', 'Refer pictures', 'As picture', 'Picture to', 'Just like the picture', 'See picture', 'See view photo', 'Fullprint', '.', 'Refer to picture', 'Clear', 'See photos', 'Just like a picture', 'As on picture', 'Faded', 'Refer photo', 'See photo', 'Natural', 'Natrual', 'Refer picture', 'Photo', 'Unknow', 'Unknown', 'As photos') THEN 'Other' end as color_group
    FROM df
    join (select category_1 || ' ' || category_2 || ' ' || category_3 as categories, COUNT(*) AS count 
        FROM df
        GROUP BY categories
        HAVING COUNT(*) > 100) c
        ON df.category_1 || ' ' || df.category_2 || ' ' || df.category_3 = c.categories
    where sold_price > 30 and score >= 4.0
""")


start0

,username,sold_price,designer,score,transaction,category_1,category_2,category_3,description,sub_title,image_count,size,color,condition,feedback_count,trans_group,image_count_group,categories,condition_encoded,feedbackcode,color_group
0,ThreepeatThrifting,165.0,Nike,5.0,97.0,Menwear,Footwear,Low Top Sneaker,Nike Dunk Panda Low Size 10 Good Condtion\nOG ...,Nike Dunk Panda Low Size 10 Good Condtion,6,Men's / US 10 / EU 43,Black,Used,58.0,0,1,Menwear Footwear Low Top Sneaker,2,3,Black
1,street_weather,89.0,Nike x Streetwear,5.0,44.0,Menwear,Outerwear,Light Jacket,Size L\nCondition 8/10\nNeed washing,NIKE SHOX VINTAGE CENTER LOGO OVERSIZE ANORAK ...,7,Men's / US L / EU 52-54 / 3,White,Gently Used,5.0,1,1,Menwear Outerwear Light Jacket,1,3,White
2,Igotsole503,180.0,Jordan Brand x Nike,5.0,49.0,Menwear,Footwear,Hi Top Sneaker,Mens size 10 / Has original box\n9/10 conditio...,Jordan 6 Electric Green size 10,11,Men's / US 10 / EU 43,Black,Gently Used,36.0,1,1,Menwear Footwear Hi Top Sneaker,1,3,Black
3,usedclothes,51.0,Nike x Stussy,5.0,2240.0,Menwear,Top,Short Sleeve T,Brand new with tags\nTagged medium fits oversi...,Increase the Peace Tee Nike x Stüssy M/L,7,Men's / US M / EU 48-50 / 2,White,New,798.0,3,1,Menwear Top Short Sleeve T,0,1,White
4,cmcctorr2546,64.0,Nike,5.0,55.0,Menwear,Footwear,Low Top Sneaker,• 8/10 Condition\nMinor dirt but can easily be...,Nike Air Max 95 Ying Yang Pack,6,Men's / US 7 / EU 40,White,Gently Used,25.0,1,1,Menwear Footwear Low Top Sneaker,1,3,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49717,BloodShoji,60.0,Calvin Klein x Calvin Klein 205W39NYC,5.0,834.0,Menwear,Top,Short Sleeve T,私の店へようこそ .\nWelcome To My Shop .\nPlease Check...,VINTAGE ‼️ Calvin Klein Spell Out Made In USA Tee,5,Men's / US L / EU 52-54 / 3,White,Used,392.0,3,2,Menwear Top Short Sleeve T,2,2,White
49718,UNTRUERELIGION,43.0,Calvin Klein x Streetwear,5.0,283.0,Menwear,Bottom,Denim,"In nice worn in condition, accurately represen...",Vintage Calvin Klein Denim Jeans,7,Men's / US 34 / EU 50,Brown,Used,98.0,3,1,Menwear Bottom Denim,2,3,Brown
49719,Markhamvtg,35.0,Calvin Klein x Streetwear,5.0,96.0,Menwear,Bottom,Short,Vintage Denim Calvin Klein Denim Shorts\nWaist...,Vintage Denim Calvin Klein Denim Shorts,4,Men's / US 34 / EU 50,Blue,Used,27.0,0,2,Menwear Bottom Short,2,3,Blue
49720,shopluckyboy,40.0,Calvin Klein x Streetwear,5.0,293.0,Menwear,Bottom,Short,Calvin Klein swim shorts\nWithout defects\nVer...,Calvin Klein black swim shorts,6,Men's / US 32 / EU 48,Black,Gently Used,111.0,3,1,Menwear Bottom Short,1,3,Black


In [85]:
start0.categories.value_counts()

Menwear Top Short Sleeve T                                                                             8743
Menwear Top Sweathirt Hoodie                                                                           7894
Menwear Bottom Denim                                                                                   4277
Menwear Outerwear Light Jacket                                                                         3594
Menwear Footwear Low Top Sneaker                                                                       2849
Menwear Top Sweater Knitwear                                                                           2350
Menwear Bottom Caual Pant                                                                              2131
Menwear Top Shirt ButtoUp                                                                              1869
Menwear Top Jerey                                                                                      1423
Menwear Outerwear Bomber    

In [57]:
start0.color.value_counts()

Black                                                        15568
White                                                         4998
Blue                                                          4666
Grey                                                          2719
Brown                                                         2093
Green                                                         1909
Red                                                           1673
Navy                                                          1584
Beige                                                          868
Multicolor                                                     798
Gray                                                           659
Yellow                                                         621
Denim                                                          558
Multi                                                          534
Pink                                                          

In [54]:
colorlist = start0.color.value_counts().index.tolist()
colorlist

['Black',
 'White',
 'Blue',
 'Grey',
 'Brown',
 'Green',
 'Red',
 'Navy',
 'Beige',
 'Multicolor',
 'Gray',
 'Yellow',
 'Denim',
 'Multi',
 'Pink',
 'Purple',
 'Orange',
 'Cream',
 'Silver',
 'Olive',
 'Camo',
 'Khaki',
 'Dark blue',
 'Navy blue',
 'Tan',
 'Gold',
 'Faded black',
 'Indigo',
 'Mix',
 'Black/white',
 'Maroon',
 'Black white',
 'Olive green',
 'Light blue',
 'Burgundy',
 'Multiple',
 'Army green',
 'Dark grey',
 'Dark green',
 'Blue denim',
 'Baby blue',
 'Tie dye',
 'Striped',
 'Blue distressed',
 'Light brown',
 'Washed black',
 'Teal',
 'Plaid',
 'Forest green',
 'Other',
 'Burgandy',
 'Multicolour',
 'Dark gray',
 'Off white',
 'Refer photo',
 'Faded blue',
 'Blue black',
 'Black/red',
 'Dark navy',
 'Blue/white',
 'Dark brown',
 'Mutli',
 'White/black',
 'Washed blue',
 'See photo',
 'Refer picture',
 'Natural',
 'Red/white',
 'Blue jean',
 'Floral',
 'Sand',
 'Black grey',
 'Stripe',
 'Charcoal',
 'Biege',
 'Peach',
 'Ivory',
 'Black gray',
 'Light grey',
 'Black r

In [51]:
primary_colors = {
    'camo': [],
    'multicolor': [],
    'blue': [],
    'grey': [],
    'brown': [],
    'green': [],
    'red': [],
    'pink': [],
    'other': [],
    'purple': [],
    'white': []
}


for color in colorlist:
    color_lower = color.lower()
    if 'camo' in color_lower:
        primary_colors['camo'].append(color)    
    elif 'multi' in color_lower:
        primary_colors['multicolor'].append(color)
    elif 'blue' in color_lower or 'navy' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['blue'].append(color)
    elif 'grey' in color_lower or 'gray' in color_lower or 'ash' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['grey'].append(color)
    elif 'brown' in color_lower or 'beige' in color_lower or 'tan' in color_lower or 'khaki' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['brown'].append(color)
    elif 'green' in color_lower or 'olive' in color_lower or 'teal' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['green'].append(color)
    elif 'red' in color_lower or 'burgundy' in color_lower or 'maroon' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['red'].append(color)
    elif 'pink' in color_lower or 'salmon' in color_lower or 'peach' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['pink'].append(color)
    elif 'purple' in color_lower or 'violet' in color_lower or 'lilac' in color_lower or 'grape' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['purple'].append(color)
    elif 'white' in color_lower or 'chalk' in color_lower:
        if len(color) >= 2 and '/' not in color and ' / ' not in color:
            primary_colors['white'].append(color)
    else:
        if len(primary_colors['other']) > 0:
            primary_colors['other'].pop()
        primary_colors['other'].append(color)

for camo, items in primary_colors.items():
    print(f"{camo}: {items}")


camo: ['Camo', 'Green camo', 'Camouflage', 'Red camo', 'Blue camo', 'Black camo', 'Woodland camo', 'Snow camo', 'Yellow camo', 'Pink camo', 'Desert camo', 'Purple camo', 'Digi camo', 'Gray camo', 'Chocolate chip camo', 'Orange camo', 'Black, green camo', 'Black red camo', 'Tiger camo', 'Quilted camo', '1st camo', 'Blue chocolate chip camo', 'City camo', 'Gray/green camo', 'Abccamo', 'Grey digital camo', 'Grey/camo', 'White camo', 'Brown camo', 'Black/white camo', 'White/camo', 'Trebark camo', 'Camouflage army', 'Reflective camo', 'Cow camo', 'Kaki camouflage', 'Star camo - black/white/olive', 'Chocolate camo', 'Reed camo brown / green / black', 'Camouflage chocolate chip', 'Bape line camo', 'Black / camo', 'Dark camo', 'Multicamo', 'Camo/neon green', 'Navy camo / balck']
multicolor: ['Multicolor', 'Multi', 'Multiple', 'Multicolour', 'Multicoloured', 'White/multi', 'Black/multi', 'Brown/multicolor', 'Multi color', 'White multi', 'Black multi', 'Multicolored', 'Black multicolor', 'Grey m

In [24]:
# sizing system is different in different category groups.

size_list = list(df['size'].values)
frequency = {}

size = pd.DataFrame(size_list)
size.value_counts()

Men's / US L / EU 52-54 / 3            13328
Men's / US M / EU 48-50 / 2            11716
Men's / US XL / EU 56 / 4               8100
ONE SIZE                                4696
Men's / US S / EU 44-46 / 1             3863
Men's / US 32 / EU 48                   2395
Men's / US XXL / EU 58 / 5              1843
Men's / US 30 / EU 46                   1619
Men's / US 34 / EU 50                   1572
Men's / US 10 / EU 43                    945
Men's / US 36 / EU 52                    806
Men's / US 9 / EU 42                     800
Men's / US 11 / EU 44                    742
Men's / US 31                            730
Men's / US 33                            708
Men's / US 9.5 / EU 42-43                464
Men's / US 10.5 / EU 43-44               446
Men's / US 12 / EU 45                    434
Men's / US 8 / EU 41                     420
Women's / M / US 6-8 / IT 42-44          403
Men's / US 28 / EU 44                    392
Men's / US 29                            381
Men's / US

In [5]:
ls = start.values[0:-1].tolist()
ls

[['  Menwear', 3],
 [' Menwear', 61052],
 [' Womenwear', 2525],
 ['Apparel Menwear', 3],
 ['College Menwear', 2],
 ['Eagle Outfitter Menwear', 1],
 ['Le Grande Bleu (L.G.B.) Womenwear', 1],
 ['Number (N)ine Menwear', 934]]

In [6]:
first_values = [sublist[0] for sublist in ls]
first_values

['  Menwear',
 ' Menwear',
 ' Womenwear',
 'Apparel Menwear',
 'College Menwear',
 'Eagle Outfitter Menwear',
 'Le Grande Bleu (L.G.B.) Womenwear',
 'Number (N)ine Menwear']

In [8]:
newlist = [s.rsplit(' ', 1)[0] for s in first_values]
newlist

[' ',
 '',
 '',
 'Apparel',
 'College',
 'Eagle Outfitter',
 'Le Grande Bleu (L.G.B.)',
 'Number (N)ine']

In [9]:
new_list = [x for x in newlist if x.strip() != '']
new_list

['Apparel',
 'College',
 'Eagle Outfitter',
 'Le Grande Bleu (L.G.B.)',
 'Number (N)ine']

In [9]:
with open('new_list2.pickle', 'wb') as f:
    pickle.dump(new_list, f)

AttributeError: 'list' object has no attribute 'isdigit'